### Pornhub Hotspots Graph

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import datetime
import json

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_colwidth", 200)

url = 'https://www.pornhub.com/view_video.php?viewkey=ph6128d2e340494'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
actions = ActionChains(driver)

def FindElements(url, driver):
    driver.get(url)
    sleep(2)
    VideoTime = driver.find_element(By.CSS_SELECTOR, 'span.mgp_total').get_attribute("innerHTML")

    player = driver.find_element(By.ID, 'player')
    hotspots = player.find_element(By.CSS_SELECTOR, 'script[type="text/javascript"]').get_attribute("innerHTML")
    return VideoTime, hotspots

def Formating(VideoTime):
    if len(VideoTime) == 2:
        VideoTime = f"00:00:{VideoTime}"

    if len(VideoTime) == 4:
        VideoTime = f"00:0{VideoTime}"

    if len(VideoTime) == 5:
        VideoTime = f"00:{VideoTime}"

    print(VideoTime)
    return VideoTime

def ToSecond(VideoTime):
    TimeDelta = pd.to_timedelta(VideoTime)
    Seconds = TimeDelta.total_seconds()
    return Seconds

def ExtractHotspotsFromString(hotspots):
    if 'hotspots' in hotspots:
        start_index_hot = hotspots.index('"hotspots"')
        start_index_url = hotspots.index(',"toprated_url"')
        hotspots_data = hotspots[start_index_hot:start_index_url]
        hotspots_data = "{" + hotspots_data + "}"
    with open('Test2.json', 'w') as data:
        hotspots_data = data.write(hotspots_data)
    with open('Test2.json', 'r') as data:
        hotspots_data = json.load(data)
        IntensityList = hotspots_data["hotspots"]
    return IntensityList

def CheckInterval(Seconds, IntensityList):
    Split = int(Seconds) / len(IntensityList)
    print(f"Interval {int(Split)}")
    print(f"Len of hotspots list {len(IntensityList)}")
    return Split

def DoDataFrame(IntensityList, Split):
    secondlist = []
    Int_IntensityList = []
    TimeDeltaList = []
    count = 0
    for i in IntensityList:
        Int_IntensityList.append(int(i))
        secondlist.append(count)
        count = int(Split) + count

    for Time in secondlist:
        Minutes = datetime.timedelta(seconds=Time)
        TimeDeltaList.append(str(Minutes))

    df = pd.DataFrame({'Time':TimeDeltaList, 'Intensity':IntensityList})
    df_sorted = df.sort_values('Intensity', ascending=False, key=lambda x: x.astype(int))
    print(f"\nNot Sorted DataFrame\n{df}\n")
    print(f"Sorted DataFrame\n{df_sorted}\n")
    return TimeDeltaList, Int_IntensityList


def Graphing(TimeDeltaList, Int_IntensityList):
    x = TimeDeltaList
    y = Int_IntensityList

    plt.plot(x, y)

    plt.xlabel('Time (00:00:00)')
    plt.ylabel('Value')
    plt.title('Replayed')

    plt.xticks(range(0, len(TimeDeltaList), 25))

    plt.show()


VideoTime, hotspots = FindElements(url, driver)
VideoTime = Formating(VideoTime)
Seconds = ToSecond(VideoTime)
IntensityList = ExtractHotspotsFromString(hotspots)
Split = CheckInterval(Seconds, IntensityList)
TimeDeltaList, Int_IntensityList = DoDataFrame(IntensityList, Split)
Graphing(TimeDeltaList, Int_IntensityList)